In [1]:
! pip install -U -q transformers
! pip install -q datasets sentence-transformers evaluate
! pip install -q accelerate -U

In [2]:
import numpy as np
import pandas as pd
import time
import re
from tqdm import notebook
import json

import seaborn as sns
import matplotlib.pyplot as plt

import os
import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModel
from transformers import AutoConfig
from transformers import get_cosine_schedule_with_warmup

from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.metrics import r2_score

from datasets import Dataset
import evaluate

from sentence_transformers import SentenceTransformer, models, util

import gc
import json

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, TextClassificationPipeline

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True

In [5]:
SEED = 97
seed_everything(SEED)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device: ', device.type)

Device:  cuda


In [7]:
def get_rt_dfs():
    dataset = load_dataset('rotten_tomatoes')
    train_df = dataset['train'].to_pandas().dropna()[['text', 'label']]
    val_df = dataset['validation'].to_pandas().dropna()[['text', 'label']]
    test_df = dataset['test'].to_pandas().dropna()[['text', 'label']]
    
    train_df.columns = ['sentence', 'label']
    val_df.columns = ['sentence', 'label']
    test_df.columns = ['sentence', 'label']

    return train_df, val_df, test_df

In [8]:
def get_fin_sent_analysis_dfs():
    df = pd.read_csv('/kaggle/input/financial-sentiment-analysis/data.csv')
    df = df[df.Sentiment != 'neutral'].reset_index(drop=True)
    df.columns = ['sentence', 'label']
    df['label'] = df['label'].apply(lambda x: 0 if x=='negative' else 1)
    
    return df

In [9]:
def get_sem_eval_2017_task4_dfs():

    dataset = load_dataset("maxmoynan/SemEval2017-Task4aEnglish", split='development')
    print(dataset)
#     train_df = dataset['train'].to_pandas().dropna()[['tweet', 'sentiment']]
#     val_df = dataset['development'].to_pandas().dropna()[['tweet', 'sentiment']]
    test_df = dataset.to_pandas().dropna()[['tweet', 'sentiment']]
    
#     train_df.columns = ['sentence', 'label']
#     val_df.columns = ['sentence', 'label']
    test_df.columns = ['sentence', 'label']
    
#     train_df = train_df[train_df.label != 1].reset_index(drop=True)
#     val_df = val_df[val_df.label != 1].reset_index(drop=True)
    test_df = test_df[test_df.label != 1].reset_index(drop=True)
    
#     train_df['label'] = train_df[train_df.label == 2].apply(lambda x: 1)
#     val_df['label'] = val_df[val_df.label == 2].apply(lambda x: 1)
    test_df['label'] = test_df[test_df.label == 2].apply(lambda x: 1)

    return test_df

In [10]:
def get_sst2_dataset_dfs():
    dataset = load_dataset('glue', 'sst2')
    train_df = dataset['train'].to_pandas().dropna()[['sentence', 'label']]
    test_df = dataset['validation'].to_pandas().dropna()[['sentence', 'label']]

    return train_df, test_df

In [11]:
def get_imdb_dataset_dfs():
    dataset = load_dataset('imdb')
    train_df = dataset['train'].to_pandas().dropna()
    train_df.columns = ['sentence', 'label']

    test_df = dataset['test'].to_pandas().dropna()
    test_df.columns = ['sentence', 'label']

    return train_df, test_df

In [12]:
def get_reddit_dfs():
    df = pd.read_csv('/kaggle/input/twitter-and-reddit-sentimental-analysis-dataset/Reddit_Data.csv')
    df = df[df.category != 0].reset_index(drop=True)
    df.columns = ['sentence', 'label']
    df['label'] = df['label'].apply(lambda x: 0 if x==-1 else 1)
    
    return df

In [13]:
def get_twitter_rd_dfs():
    df = pd.read_csv('/kaggle/input/twitter-and-reddit-sentimental-analysis-dataset/Twitter_Data.csv').dropna()
    df = df[df.category != 0].sample(frac=0.5).reset_index(drop=True)
    df.columns = ['sentence', 'label']
    df['label'] = df['label'].apply(lambda x: 0 if x==-1 else 1)
    
    return df

In [14]:
def get_kindle_reviews_dfs():
    df = pd.read_csv('/kaggle/input/amazon-kindle-book-review-for-sentiment-analysis/preprocessed_kindle_review .csv')
    df = df[['reviewText', 'rating']].reset_index(drop=True)
    df.columns = ['sentence', 'label']
    df['label'] = df['label'].apply(lambda x: 0 if x<3 else 1)
    return df

In [15]:
def get_tweet_eval_dfs():
    dataset = load_dataset('tweet_eval', 'sentiment')
    train_df = dataset['train'].to_pandas().dropna()
    val_df = dataset['validation'].to_pandas().dropna()
    test_df = dataset['test'].to_pandas().dropna()

    train_df.columns = ['sentence', 'label']
    val_df.columns = ['sentence', 'label']
    test_df.columns = ['sentence', 'label']

    train_df = train_df[train_df.label != 2].reset_index(drop=True)
    val_df = val_df[val_df.label != 2].reset_index(drop=True)
    test_df = test_df[test_df.label != 2].reset_index(drop=True)

    return train_df, val_df, test_df

In [16]:
def get_yelp_polarity_dfs():
    dataset = load_dataset('yelp_polarity')
    test_df = dataset['test'].to_pandas().dropna()
    test_df.columns = ['sentence', 'label']

    return test_df

In [17]:
def get_amazon_polarity_dfs():
    dataset = load_dataset('amazon_polarity')
    test_df = dataset['test'].to_pandas().dropna()

    test_df = test_df[['content', 'label']].reset_index(drop=True)

    test_df.columns = ['sentence', 'label']

    return test_df

In [18]:
test_dfs = {}

In [19]:
train_df, val_df, test_df = get_rt_dfs()
test_dfs['rottent_tomatoes'] = test_df

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Dataset rotten_tomatoes_movie_review downloaded and prepared to /root/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
# test_df = get_amazon_polarity_dfs()
# test_dfs['amazon'] = test_df

test_df = get_yelp_polarity_dfs()
test_dfs['yelp'] = test_df

Generating train split:   0%|          | 0/560000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/38000 [00:00<?, ? examples/s]

Dataset yelp_polarity downloaded and prepared to /root/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/14f90415c754f47cf9087eadac25823a395fef4400c7903c5897f55cfaaa6f61. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
train_df, val_df, test_df = get_tweet_eval_dfs()
test_dfs['tweet_eval'] = test_df

test_df = get_kindle_reviews_dfs()
test_dfs['kindle'] = test_df

test_df = get_twitter_rd_dfs()
test_dfs['twitter_rd'] = test_df

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45615 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12284 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [22]:
test_df = get_reddit_dfs()
test_dfs['reddit'] = test_df

train_df, test_df = get_imdb_dataset_dfs()
test_dfs['imdb'] = test_df

train_df, test_df = get_sst2_dataset_dfs()
test_dfs['sst2'] = test_df

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [23]:
# test_df = get_sem_eval_2017_task4_dfs()
# test_dfs['sem_eval_2017'] = test_df

test_df = get_fin_sent_analysis_dfs()
test_dfs['fin_sent'] = test_df

In [24]:
DATASET_NAME = "sst2"

In [25]:
MODEL_NAME = "bert-base-uncased"
MAX_LEN = 128
# TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 128
# SAVE_MODEL_PATH = "/content/drive/MyDrive/Representative Subset/Models"
# SAVE_LOGS_PATH = "/content/drive/MyDrive/Representative Subset/Logs"
# LEARNING_RATE = 5e-5
# EPOCHS = 3

In [26]:
SAVED_METRICS_PATH = f"/kaggle/input/sst2-models-bert/SST2-Models-BERT/run-metrics.json"
SAVED_MODELS_PATH = f"/kaggle/input/sst2-models-bert/SST2-Models-BERT/Subset-Models"

In [27]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [28]:
# Funtion to tokenize data
def tokenize_dataset(data):
    return tokenizer(data["sentence"],
                     max_length=MAX_LEN,
                     truncation=True,
                     padding="max_length"
                     )

In [29]:
def get_hf_dataset(df):
    data = Dataset.from_pandas(df)
    dataset = data.map(tokenize_dataset)
    return dataset

In [30]:
# Function to compute the metric
def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [31]:
with open(SAVED_METRICS_PATH, 'r') as f:
    data = json.load(f)

model_sizes = data['variance_metrics']['subset_sizes']
model_sizes

[5387, 10774, 16161, 21548, 26935, 32322, 37709, 43096, 48483, 53870]

In [32]:
temp_training_args = TrainingArguments(
    output_dir='./results',
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    report_to='none'
)

In [33]:
def save_metrics(filePath, evaluation_metrics, model_sizes):
    metrics = {'evaluation_metrics': evaluation_metrics, 'model_sizes': model_sizes}
    with open(filePath, "w") as outfile:
        json.dump(metrics, outfile)

In [34]:
for key, test_df in test_dfs.items():
    if key != DATASET_NAME:
        TEST_DATASET_NAME = key
        print(f"Testing for dataset: {TEST_DATASET_NAME} with {len(test_df)} samples.")
        test_dataset = get_hf_dataset(test_df)
        evaluation_metrics = []
        for size in model_sizes:
            print(f"Loading model from: {SAVED_MODELS_PATH}/model-{size}")
            model = AutoModelForSequenceClassification.from_pretrained(f"{SAVED_MODELS_PATH}/model-{size}").to(device)
            trainer = Trainer(
                model=model,                         # the instantiated 🤗 Transformers model to be trained
                args=temp_training_args,                  # training arguments, defined above
                compute_metrics=compute_metrics     # the callback that computes metrics of interest
            )

            # Evaluate the model
            eval_result = trainer.evaluate(eval_dataset=test_dataset)
            print(eval_result)
            evaluation_metrics.append(eval_result)

            del model
            gc.collect()
            
        save_metrics(f"/kaggle/working/eval-metrics-{TEST_DATASET_NAME}.json", evaluation_metrics, model_sizes)

Testing for dataset: rottent_tomatoes with 1066 samples.


/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


  0%|          | 0/1066 [00:00<?, ?ex/s]

Loading model from: /kaggle/input/sst2-models-bert/SST2-Models-BERT/Subset-Models/model-5387


{'eval_loss': 0.37482669949531555, 'eval_accuracy': 0.8424015009380863, 'eval_runtime': 7.3482, 'eval_samples_per_second': 145.07, 'eval_steps_per_second': 1.225}
Loading model from: /kaggle/input/sst2-models-bert/SST2-Models-BERT/Subset-Models/model-10774


...

{'eval_loss': 0.7479416131973267, 'eval_accuracy': 0.6242625368731564, 'eval_runtime': 12.2364, 'eval_samples_per_second': 221.633, 'eval_steps_per_second': 1.798}
Loading model from: /kaggle/input/sst2-models-bert/SST2-Models-BERT/Subset-Models/model-37709


{'eval_loss': 0.8969504833221436, 'eval_accuracy': 0.5792772861356932, 'eval_runtime': 12.234, 'eval_samples_per_second': 221.677, 'eval_steps_per_second': 1.798}
Loading model from: /kaggle/input/sst2-models-bert/SST2-Models-BERT/Subset-Models/model-43096


{'eval_loss': 0.6516368985176086, 'eval_accuracy': 0.6980088495575221, 'eval_runtime': 12.4315, 'eval_samples_per_second': 218.156, 'eval_steps_per_second': 1.77}
Loading model from: /kaggle/input/sst2-models-bert/SST2-Models-BERT/Subset-Models/model-48483


{'eval_loss': 0.9179368019104004, 'eval_accuracy': 0.6670353982300885, 'eval_runtime': 12.207, 'eval_samples_per_second': 222.167, 'eval_steps_per_second': 1.802}
Loading model from: /kaggle/input/sst2-models-bert/SST2-Models-BERT/Subset-Models/model-53870


{'eval_loss': 0.9019126296043396, 'eval_accuracy': 0.7035398230088495, 'eval_runtime': 12.3625, 'eval_samples_per_second': 219.373, 'eval_steps_per_second': 1.78}
